In [1]:
import pandas as pd
import numpy as np
!pip install lxml


     |████████████████████████████████| 5.8MB 5.3MB/s eta 0:00:01


Data Section(Cleaning/Preparation)

In [2]:
df1=pd.read_html('https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations')

In [3]:
df2=df1[0]

In [4]:
df2.head()

,Name,Latitude,Longitude,Platform / Entrance,Collected By,Collected On,Line,Step free
0,Acton Town,51.502500,-0.278126,Platform,User:Gagravarr,24/11/06,"District, Piccadilly",NaN
1,Acton Central,51.50883531,-0.263033174,Entrance,User:Firefishy,08/05/2007,London Overground,NaN
2,Acton Central,51.50856013,-0.262879534,Platform,User:Firefishy,08/05/2007,London Overground,NaN
3,Aldgate,51.51394,-0.07537,Aldgate High Street entrance,User:Morwen,28/4/2007,Metropolitan,No
4,Aldgate East,51.51514,-0.07178,Entrance,User:Parsingphase,(2006),"District, Hammersmith & City",NaN


In [5]:
df2.drop(['Platform / Entrance','Collected By','Collected On','Step free'],axis=1,inplace=True)

In [6]:
df2.head()

,Name,Latitude,Longitude,Line
0,Acton Town,51.502500,-0.278126,"District, Piccadilly"
1,Acton Central,51.50883531,-0.263033174,London Overground
2,Acton Central,51.50856013,-0.262879534,London Overground
3,Aldgate,51.51394,-0.07537,Metropolitan
4,Aldgate East,51.51514,-0.07178,"District, Hammersmith & City"


In [7]:
df2.set_index('Name')

,Latitude,Longitude,Line
Name,,,
Acton Town,51.502500,-0.278126,"District, Piccadilly"
Acton Central,51.50883531,-0.263033174,London Overground
Acton Central,51.50856013,-0.262879534,London Overground
Aldgate,51.51394,-0.07537,Metropolitan
Aldgate East,51.51514,-0.07178,"District, Hammersmith & City"
...,...,...,...
Wimbledon,51.42200,-0.20544,District
Wimbledon Park,51.43391,-0.19864,District
Wood Green,51.59709,-0.10939,Piccadilly


In [9]:
#df2.drop(["Canada Water"],inplace=True ) 
  

In [10]:
df2= df2[df2.Name != 'Canada Water']

In [11]:
df2.shape

(301, 4)

In [12]:
df2['Latitude'] = pd.to_numeric(df2['Latitude'])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df2['Longitude'] = pd.to_numeric(df2['Longitude'])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df2.head()

,Name,Latitude,Longitude,Line
0,Acton Town,51.502500,-0.278126,"District, Piccadilly"
1,Acton Central,51.508835,-0.263033,London Overground
2,Acton Central,51.508560,-0.262880,London Overground
3,Aldgate,51.513940,-0.075370,Metropolitan
4,Aldgate East,51.515140,-0.071780,"District, Hammersmith & City"


In [176]:
df2.shape

(301, 3)

In [65]:
#df11=pd.read_html('https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations')

In [15]:
df3=pd.read_html('https://en.wikipedia.org/wiki/List_of_London_railway_stations')

In [16]:
df4=df3[1]

In [17]:
df4.rename(columns={'Station': 'StationName'}, inplace=True)


In [20]:
df4.head()

,Rank,StationName,Yearly passengers
0,1,Waterloo,98442742
1,2,Victoria,81356330
2,3,Liverpool Street,63004002
3,4,London Bridge,56442044
4,5,Euston,41911706


In [21]:
print('The dataframe has {} line and {} station name.'.format(
        len(df2['Line'].unique()),
        df2.shape[0]
    )
)

The dataframe has 53 line and 301 station name.


Data Exploration And Visualization

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

In [23]:
!pip install geopy  # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 112kB 24.6MB/s eta 0:00:01
Libraries imported.


In [25]:
address = 'London, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [26]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, line, name in zip(df2['Latitude'], df2['Longitude'], df2['Name'], df2['Line']):
    label = '{}, {}'.format(name, line)
    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Simplifying  the above map,segment and cluster only the neighborhoods in and around the Piccadilly line area.

Slicing the original dataframe and creating a new dataframe of the Piccadilly data.

In [27]:
Piccadilly_data = df2[df2['Line'] == 'Piccadilly'].reset_index(drop=True)
Piccadilly_data.head(15)


,Name,Latitude,Longitude,Line
0,Alperton,51.540970,-0.300610,Piccadilly
1,Arnos Grove,51.616250,-0.133550,Piccadilly
2,Arsenal,51.558470,-0.105610,Piccadilly
3,Boston Manor,51.495371,-0.325573,Piccadilly
4,Bounds Green,51.607000,-0.124180,Piccadilly
5,Caledonian Road,51.548500,-0.118020,Piccadilly
6,Cockfosters,51.651170,-0.148110,Piccadilly
7,Covent Garden,51.513080,-0.124270,Piccadilly
8,Hatton Cross,51.466988,-0.423035,Piccadilly
9,Heathrow Terminals 1-2-3,51.471290,-0.452744,Piccadilly


In [28]:
address = 'Piccadilly, London'

geolocator = Nominatim(user_agent="foursquare_agent")

location = geolocator.geocode(address)

latitude = location.latitude

longitude = location.longitude

print('The geograpical coordinate of Piccadilly are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Piccadilly are 51.5031159, -0.1519364.


In [29]:
# create map of Piccadilly using latitude and longitude values
map_Piccadilly = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Piccadilly_data['Latitude'], Piccadilly_data['Longitude'], Piccadilly_data['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Piccadilly)  
    
map_Piccadilly

Foursquare Credentials :

In [30]:
CLIENT_ID = 'UWULHDGZWTE05NWFHPQ5GYMTNXCTOXVNFSSFFWEZNERKO0TN' # your Foursquare ID
CLIENT_SECRET = 'YI3CYQVDOQJWWYK3QJKW5ZJZALS5M31XDVBTGONJISXIAHZZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UWULHDGZWTE05NWFHPQ5GYMTNXCTOXVNFSSFFWEZNERKO0TN
CLIENT_SECRET:YI3CYQVDOQJWWYK3QJKW5ZJZALS5M31XDVBTGONJISXIAHZZ


Exploring the area around the Piccadilly line

Choosing the area around the 'Arsenal' station

In [31]:
Piccadilly_data.loc[2, 'Name']

'Arsenal'

In [32]:
name_latitude = Piccadilly_data.loc[2, 'Latitude'] # station name latitude value
name_longitude = Piccadilly_data.loc[2, 'Longitude'] # station name longitude value

station_name = Piccadilly_data.loc[2, 'Name'] # station name

print('Latitude and longitude values of {} are {}, {}.'.format(station_name, 
                                                               name_latitude, 
                                                               name_longitude))

Latitude and longitude values of Arsenal are 51.55847, -0.10561.


Now, let's get the top 100 venues that are around the Arsenal station, within a radius of 500 meters.

First, let's create the GET request URL and name the   URL as **url**.

In [33]:

LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    name_latitude, 
    name_longitude, 
    radius, 
    LIMIT)
url # display URL





'https://api.foursquare.com/v2/venues/explore?&client_id=UWULHDGZWTE05NWFHPQ5GYMTNXCTOXVNFSSFFWEZNERKO0TN&client_secret=YI3CYQVDOQJWWYK3QJKW5ZJZALS5M31XDVBTGONJISXIAHZZ&v=20180605&ll=51.55847,-0.10561&radius=500&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9fdeeabae9a2001b47e2f0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Islington',
  'headerFullLocation': 'Islington, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 51.5629700045,
    'lng': -0.09838547161098131},
   'sw': {'lat': 51.5539699955, 'lng': -0.11283452838901868}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd2d84541b9ef3b0cc4fee5',
       'name': 'The Arsenal Store',
       'location': {'address': '75 Drayton Park',
        'crossStreet': 'Highbury House',
        'lat': 51.55669516351197,
        'lng': -0.10609761727323813,
        'labeledLatLngs': [{'l

Extracting the venue categorywith the get_category_type function

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning  the json and structuring it into a pandas dataframe.


In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Arsenal Store,Clothing Store,51.556695,-0.106098
1,DW Fitness First,Gym / Fitness Center,51.558208,-0.102262
2,Emirates Stadium,Soccer Stadium,51.555247,-0.108361
3,The North Bank,Soccer Stadium,51.555700,-0.108202
4,East Stand,Soccer Stadium,51.554861,-0.107310


In [222]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

33 venues were returned by Foursquare.


#### Exploring areas around *Piccadilly line* 

Creating  a function to repeat the same process to all the areas around Piccadilly line 

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

####   The code to run the above function on each neighborhood and create a new dataframe called *Piccadilly_venues*.

In [40]:
Piccadilly_venues = getNearbyVenues(names=Piccadilly_data['Name'],
                                   latitudes=Piccadilly_data['Latitude'],
                                   longitudes=Piccadilly_data['Longitude']
                                  )


Alperton
Arnos Grove
Arsenal
Boston Manor
Bounds Green
Caledonian Road
Cockfosters
Covent Garden
Hatton Cross
Heathrow Terminals 1-2-3
Heathrow Terminal 4
Holloway Road
Hounslow Central
Hounslow East
Hounslow West
Hyde Park Corner
Knightsbridge
Manor House
North Ealing
Northfields
Oakwood
Osterley
Park Royal
Russell Square
South Ealing
South Harrow
Southgate
Sudbury Hill
Sudbury Town
Turnpike Lane
Wood Green


In [41]:
print(Piccadilly_venues.shape)
Piccadilly_venues.head()

(909, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alperton,51.54097,-0.30061,The Gym,51.540819,-0.298715,Gym / Fitness Center
1,Alperton,51.54097,-0.30061,Sainsbury's,51.538431,-0.302540,Supermarket
2,Alperton,51.54097,-0.30061,Maru Bhajias,51.543873,-0.297200,Indian Restaurant
3,Alperton,51.54097,-0.30061,Subway,51.541707,-0.297996,Sandwich Place
4,Alperton,51.54097,-0.30061,East Pan Asian Restaurant,51.537700,-0.301996,Asian Restaurant


In [42]:
Piccadilly_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alperton,10,10,10,10,10,10
Arnos Grove,7,7,7,7,7,7
Arsenal,32,32,32,32,32,32
Boston Manor,7,7,7,7,7,7
Bounds Green,16,16,16,16,16,16
Caledonian Road,26,26,26,26,26,26
Cockfosters,24,24,24,24,24,24
Covent Garden,92,92,92,92,92,92
Hatton Cross,11,11,11,11,11,11


In [227]:
print('There are {} uniques categories.'.format(len(Piccadilly_venues['Venue Category'].unique())))

There are 189 uniques categories.


Analyzing each neighborhoods

In [43]:
# one hot encoding
Piccadilly_onehot = pd.get_dummies(Piccadilly_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Piccadilly_onehot['Neighborhood'] = Piccadilly_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Piccadilly_onehot.columns[-1]] + list(Piccadilly_onehot.columns[:-1])
Piccadilly_onehot = Piccadilly_onehot[fixed_columns]

Piccadilly_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Bistro,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Bulgarian Restaurant,Burger Joint,Bus Station,Bus Stop,Café,Campground,Canal Lock,Casino,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Neighborhood,Newsstand,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Reservoir,Restaurant,Roof Deck,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alperton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alperton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alperton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [44]:
Piccadilly_onehot.shape

(909, 195)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
Piccadilly_grouped = Piccadilly_onehot.groupby('Neighborhood').mean().reset_index()
Piccadilly_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Bistro,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Bulgarian Restaurant,Burger Joint,Bus Station,Bus Stop,Café,Campground,Canal Lock,Casino,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Newsstand,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Reservoir,Restaurant,Roof Deck,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Women's Store
0,Alperton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.200000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000,0.

In [46]:
Piccadilly_grouped.shape

(31, 195)

In [47]:
Piccadilly_grouped.rename(columns={'Neighborhood':'Name'},inplace=True)


In [48]:
Piccadilly_grouped.head()

,Name,Yoga Studio,Accessories Store,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Bistro,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Building,Bulgarian Restaurant,Burger Joint,Bus Station,Bus Stop,Café,Campground,Canal Lock,Casino,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Newsstand,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Reservoir,Restaurant,Roof Deck,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Women's Store
0,Alperton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.00000,0.0,0.000000,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.1,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.20000,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.2,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Arnos Grove,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.285714,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.142857,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0

Let's print each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in Piccadilly_grouped['Name']:
    print("----"+hood+"----")
    temp = Piccadilly_grouped[Piccadilly_grouped['Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alperton----
                  venue  freq
0           Supermarket   0.2
1     Indian Restaurant   0.2
2  Gym / Fitness Center   0.2
3           Bridal Shop   0.1
4      Asian Restaurant   0.1


----Arnos Grove----
                venue  freq
0            Bus Stop  0.29
1       Grocery Store  0.29
2  Chinese Restaurant  0.14
3            Beer Bar  0.14
4                Park  0.14


----Arsenal----
                  venue  freq
0                   Pub  0.12
1        Soccer Stadium  0.12
2                  Park  0.09
3  Gym / Fitness Center  0.09
4            Sports Bar  0.09


----Boston Manor----
            venue  freq
0             Bar  0.14
1        Bus Stop  0.14
2  Breakfast Spot  0.14
3      Canal Lock  0.14
4            Farm  0.14


----Bounds Green----
          venue  freq
0   Coffee Shop  0.19
1           Pub  0.12
2        Bakery  0.06
3    Campground  0.06
4  Noodle House  0.06


----Caledonian Road----
                 venue  freq
0  Rental Car Location  0.12
1        

Converting into a Dataframe

First, let's write a function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Name'] = Piccadilly_grouped['Name']

for ind in np.arange(Piccadilly_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Piccadilly_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alperton,Gym / Fitness Center,Indian Restaurant,Supermarket,Fast Food Restaurant,Asian Restaurant,Bridal Shop,Sandwich Place,Exhibit,French Restaurant,Food Truck
1,Arnos Grove,Grocery Store,Bus Stop,Beer Bar,Park,Chinese Restaurant,Women's Store,French Restaurant,Food Truck,Food Court,Fish & Chips Shop
2,Arsenal,Soccer Stadium,Pub,Sports Bar,Gym / Fitness Center,Park,Café,Clothing Store,Fish & Chips Shop,Food Truck,Pizza Place
3,Boston Manor,Farm,Chinese Restaurant,Bus Stop,Canal Lock,Bar,Theater,Breakfast Spot,English Restaurant,Ethiopian Restaurant,Fruit & Vegetable Store
4,Bounds Green,Coffee Shop,Pub,Gourmet Shop,Grocery Store,Tennis Court,Café,Breakfast Spot,Campground,Convenience Store,Noodle House


# Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

Piccadilly_grouped_clustering = Piccadilly_grouped.drop('Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Piccadilly_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Piccadilly_merged = Piccadilly_data

# merge Piccadilly_grouped with Piccadilly_data to add latitude/longitude for each neighborhood
Piccadilly_merged = Piccadilly_merged.join(neighborhoods_venues_sorted.set_index('Name'), on='Name')

Piccadilly_merged.head() # check the last columns!

,Name,Latitude,Longitude,Line,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alperton,51.540970,-0.300610,Piccadilly,0,Gym / Fitness Center,Indian Restaurant,Supermarket,Fast Food Restaurant,Asian Restaurant,Bridal Shop,Sandwich Place,Exhibit,French Restaurant,Food Truck
1,Arnos Grove,51.616250,-0.133550,Piccadilly,4,Grocery Store,Bus Stop,Beer Bar,Park,Chinese Restaurant,Women's Store,French Restaurant,Food Truck,Food Court,Fish & Chips Shop
2,Arsenal,51.558470,-0.105610,Piccadilly,1,Soccer Stadium,Pub,Sports Bar,Gym / Fitness Center,Park,Café,Clothing Store,Fish & Chips Shop,Food Truck,Pizza Place
3,Boston Manor,51.495371,-0.325573,Piccadilly,4,Farm,Chinese Restaurant,Bus Stop,Canal Lock,Bar,Theater,Breakfast Spot,English Restaurant,Ethiopian Restaurant,Fruit & Vegetable Store
4,Bounds Green,51.607000,-0.124180,Piccadilly,1,Coffee Shop,Pub,Gourmet Shop,Grocery Store,Tennis Court,Café,Breakfast Spot,Campground,Convenience Store,Noodle House


# Visualizing the clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Piccadilly_merged['Latitude'], Piccadilly_merged['Longitude'], Piccadilly_merged['Name'], Piccadilly_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 1

In [55]:
Piccadilly_merged.loc[Piccadilly_merged['Cluster Labels'] == 0, Piccadilly_merged.columns[[1] + list(range(5, Piccadilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.54097,Gym / Fitness Center,Indian Restaurant,Supermarket,Fast Food Restaurant,Asian Restaurant,Bridal Shop,Sandwich Place,Exhibit,French Restaurant,Food Truck


Cluster 2

In [56]:
Piccadilly_merged.loc[Piccadilly_merged['Cluster Labels'] == 1, Piccadilly_merged.columns[[1] + list(range(5, Piccadilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,51.558470,Soccer Stadium,Pub,Sports Bar,Gym / Fitness Center,Park,Café,Clothing Store,Fish & Chips Shop,Food Truck,Pizza Place
4,51.607000,Coffee Shop,Pub,Gourmet Shop,Grocery Store,Tennis Court,Café,Breakfast Spot,Campground,Convenience Store,Noodle House
5,51.548500,Café,Rental Car Location,Park,Theater,Coffee Shop,Chinese Restaurant,Restaurant,Pub,Yoga Studio,Breakfast Spot
6,51.651170,Italian Restaurant,Café,Turkish Restaurant,Greek Restaurant,Platform,Bakery,Coffee Shop,Fish & Chips Shop,French Restaurant,Fruit & Vegetable Store
7,51.513080,Theater,Coffee Shop,Sushi Restaurant,Cocktail Bar,Hotel,Burger Joint,Music Store,Dessert Shop,Steakhouse,Restaurant
8,51.466988,Hotel,Convenience Store,Pub,Rental Car Location,Bus Station,Fast Food Restaurant,Trail,Gym / Fitness Center,English Restaurant,Metro Station
9,51.471290,Airport Lounge,Coffee Shop,Department Store,Clothing Store,Restaurant,Pharmacy,Grocery Store,Pub,Train Station,Fast Food Restaurant
10,51.459478,Airport Lounge,Coffee Shop,Hotel,Cosmetics Shop,Airport Service,Italian Restaurant,Bookstore,Airport Terminal,Sandwich Place,Scenic Lookout
11,51.552900,Café,Soccer Stadium,Pub,Coffee Shop,Ethiopian Restaurant,Gym / Fitness Center,Women's Store,Malay Restaurant,Seafood Restaurant,Restaurant
12,51.471290,Clothing Store,Hotel,Coffee Shop,Grocery Store,Fast Food Restaurant,Pharmacy,Video Game Store,Sandwich Place,Bakery,Sporting Goods Shop


Cluster 3

In [57]:
Piccadilly_merged.loc[Piccadilly_merged['Cluster Labels'] == 2, Piccadilly_merged.columns[[1] + list(range(5, Piccadilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,51.556998,Hotel,Train Station,Metro Station,Grocery Store,Women's Store,Exhibit,French Restaurant,Food Truck,Food Court,Fish & Chips Shop


Cluster 4

In [58]:
Piccadilly_merged.loc[Piccadilly_merged['Cluster Labels'] == 3, Piccadilly_merged.columns[[1] + list(range(5, Piccadilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,51.473540,Indian Restaurant,Bus Stop,Asian Restaurant,Hostel,Grocery Store,Metro Station,Dessert Shop,Convenience Store,Coffee Shop,Supermarket
14,51.473689,Indian Restaurant,Hotel,Bus Stop,Platform,Pharmacy,Asian Restaurant,Park,Grocery Store,Event Space,Food Court


In [59]:
Piccadilly_merged.loc[Piccadilly_merged['Cluster Labels'] == 4, Piccadilly_merged.columns[[1] + list(range(5, Piccadilly_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,51.616250,Grocery Store,Bus Stop,Beer Bar,Park,Chinese Restaurant,Women's Store,French Restaurant,Food Truck,Food Court,Fish & Chips Shop
3,51.495371,Farm,Chinese Restaurant,Bus Stop,Canal Lock,Bar,Theater,Breakfast Spot,English Restaurant,Ethiopian Restaurant,Fruit & Vegetable Store
19,51.499856,Park,Café,Bus Stop,Italian Restaurant,Brewery,Kebab Restaurant,Grocery Store,French Restaurant,Diner,Deli / Bodega
20,51.647390,Greek Restaurant,Grocery Store,Indian Restaurant,Paper / Office Supplies Store,Bus Stop,Café,Golf Course,Metro Station,Chinese Restaurant,Bakery
21,51.481569,Bus Stop,Convenience Store,Thai Restaurant,Hockey Field,Metro Station,Coffee Shop,Chinese Restaurant,Sandwich Place,Bus Station,Bookstore
24,51.500801,Pizza Place,Coffee Shop,Indian Restaurant,Pub,Gay Bar,Breakfast Spot,Mediterranean Restaurant,Bus Stop,Grocery Store,Bakery
28,51.550930,Indian Restaurant,Pub,Fast Food Restaurant,Bus Stop,Middle Eastern Restaurant,Train Station,Park,Metro Station,Women's Store,Falafel Restaurant
